### In which counties is the mix of party registrations most different from state-wide averages?

In [ ]:
%%sql
Select t.countycode, t.partycode, (count(t.registrantid)/s.count_value_sum)/w.percentage as county_v_natl_pct
FROM voter_grand t
JOIN (SELECT countycode, count(registrantid) AS count_value_sum 
FROM voter_grand
GROUP BY countycode) s
ON s.countycode = t.countycode 
JOIN (SELECT u.partycode, count(u.registrantid)/v.count_value_sum as percentage
FROM voter_grand u
JOIN
(SELECT count(registrantid) AS count_value_sum 
FROM voter_grand) v
WHERE u.partycode IN ("DEM", "GRN", "LIB", "REP")
GROUP BY u.partycode, v.count_value_sum) w
ON w.partycode = t.partycode
WHERE t.partycode IN ("DEM", "REP", "GRN", "LIB")
GROUP BY t.countycode, t.partycode, s.count_value_sum, w.percentage
ORDER BY county_v_natl_pct DESC

The most Republican counties are: Modoc, Lassen, and Shasta.  The most Liberterian: Placer, Calaveras, and Sierra.  Most Green: Mendocino, Humboldt, and Trinity.  Most Democratic: San Francisco, Alameda, and Santa Cruz.

### In which counties was turnout (number who voted as a % of those registered) highest and lowest?

In [ ]:
%%sql
SELECT a.countycode, count(a.registrantid) as registereds, b.voters, b.voters/count(a.registrantid) as pct_voted
FROM voter_grand_royale a
JOIN 
(SELECT countycode, count(registrantid) as voters
FROM voter_grand_royale
WHERE 2016ge_method IS NOT NULL
GROUP BY countycode) b
on a.countycode = b.countycode
GROUP BY a.countycode, b.voters
ORDER BY pct_voted DESC
LIMIT 4


The counties with the highest turnout were: Marin, Glenn, and Sonoma.  Weakest turnout were: Imperial, Kings, and Fresno.

### How did statewide turnout (numbers of votes cast compard to number of registered voters) vary by party?

In [ ]:
%%sql
Select a.partycode, count(a.registrantid) as voternum, b.count_value_sum as registerednum, count(a.registrantid)/b.count_value_sum as turnoutpct
FROM voter_grand_royale a
JOIN 
(SELECT partycode, count(registrantid) as count_value_sum
FROM voter_grand_royale
WHERE partycode IN ("DEM", "GRN", "LIB", "REP", "NPP")
GROUP BY partycode) b
on a.partycode = b.partycode 
WHERE a.partycode IN ("DEM", "GRN", "LIB", "REP", "NPP") and a.2016ge_method IS NOT NULL
GROUP BY a.partycode, b.count_value_sum

Registered Republicans turned out in the strongest numbers and registrants noting "no party affiliation" were the least engaged.

### What is the breakdown by party affiliation of "first-time voters" in 2016 presidential election?

In [ ]:
%%sql
select count(v.registrantid) as votercount,  v.partycode
FROM (select registrantid, cast(to_date(from_unixtime(unix_timestamp(dob, 'yyyy-MM-dd'))) as date) as date, countycode, partycode
from voter_grand_royale
where 2016ge_method IS NOT NULL AND 2014ge_method IS NULL and 2012ge_method IS NULL and 2010ge_method IS NULL and 2008ge_method IS NULL
) v
where v.date < 1987-02-02 and partycode in ("DEM", "REP", "LIB", "GRN", "NPP")
group by v.partycode
sort by votercount DESC

Most people who voted in 2016 after sitting out 2008, 10, 12 despite being eligible, were registered Democrats. More than twice as many as were Republican in fact.

### What is the gender breakdown of registrants by party?

In [ ]:
%%sql
SELECT a.partycode, a.tot_males/b.tot_registered as pct_male
FROM 
(SELECT partycode, count(registrantid) as tot_males
FROM 
voter_grand_royale
WHERE gender = "M" and partycode in ("DEM", "REP", "LIB", "GRN")
GROUP BY partycode) a
JOIN
(SELECT partycode, count(registrantid)as tot_registered
FROM voter_grand_royale
WHERE gender IN ("M", "F") and partycode in ("DEM", "REP", "LIB", "GRN")
GROUP BY partycode) b
on a.partycode = b.partycode

The Liberterian party boasts the highest % of male registrants and the Democratic party the least.